In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive
%ls

/content/drive/MyDrive
'Bobby p3 draft.gdoc'                      zp2019_hw1.ipynb
'Bobby P3.gdoc'                            zp2019_hw1.pdf
'Bobby Peng Farewell .gdoc'                zp2019_hw2.ipynb
'Colab Notebooks'/                         zp2019_hw2.pdf
 lazydata/                                 zp2019_hw3.pdf
 leftImg8bit_trainvaltest/                'zp2019_hw4 (1).pdf'
 NYU_SR_TUNF.pdf                          'zp2019_hw4_part1 (1).py'
 NYU_Unofficial_Transcript.pdf             zp2019_hw4_part1.py
'P2 Draft Bobby Peng.gdoc'                'zp2019_hw4_part2 (1).py'
'P2EX2 (1).gdoc'                           zp2019_hw4_part2.py
 P2EX2.gdoc                                zp2019_hw4.pdf
'PRACTICE-UA1 EXAM 1.gdoc'                'zp2019_hw5 (1).ipynb'
'PRACTICE-UA1 EXAM 1.pdf'                 'zp2019_hw5 (2).ipynb'
'Reflective writing Bobby Peng.gdoc'      'zp2019_hw5 (3).ipynb'
 Resume_Bobby_Peng.pdf                    'zp2019_hw5_extra_credit (1).ipynb'
'Screen Shot 2022-11-12 at 

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import cv2
import pickle as pkl
import torch.nn.functional as F
import torch.optim as optim

In [4]:
class LazyLoadDataset(Dataset):
  def __init__(self, path, train=True, transform=None):
    self.transform = transform
    path = path + ('train/' if train else 'test/')
    self.pathX = path + 'X/'
    self.pathY = path + 'Y/'
    self.data = os.listdir(self.pathX)

  def __getitem__(self, idx):
    f = self.data[idx]
    img0 = cv2.imread(self.pathX+f+'/rgb/0.png')
    img1 = cv2.imread(self.pathX+f+'/rgb/1.png')
    img2 = cv2.imread(self.pathX+f+'/rgb/2.png')
    #merge images into one
    img = img0+img1+img2
    if self.transform is not None:
      img = self.transform(img)
    depth = np.load(self.pathX+f+'/depth.npy')
    field_id = pkl.load(open(self.pathX+f+'/field_id.pkl', 'rb'))
    Y = np.load(self.pathY+f+'.npy')
    return (img, depth, field_id), Y
  
  def getTestitem(self, idx):
    f = self.data[idx]
    img0 = cv2.imread(self.pathX+f+'/rgb/0.png')
    img1 = cv2.imread(self.pathX+f+'/rgb/1.png')
    img2 = cv2.imread(self.pathX+f+'/rgb/2.png')
    #merge images into one
    img = img0+img1+img2
    if self.transform is not None:
      img = self.transform(img)
    depth = np.load(self.pathX+f+'/depth.npy')
    field_id = pkl.load(open(self.pathX+f+'/field_id.pkl', 'rb'))
    return img, depth, field_id

  def __len__(self):
    return len(self.data)

In [5]:
my_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [6]:
train_data = LazyLoadDataset('./lazydata/', transform=my_transform)
test_data = LazyLoadDataset('./lazydata/', train=False, transform=my_transform)

In [7]:
train_loader = DataLoader(train_data, batch_size=12, shuffle=True)


In [31]:
train_it = iter(train_loader)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:

def train(epoch, model, optimizer):

    model.train()
    for batch_idx, ((img, depth, field_id), Y) in enumerate(train_loader):
        for batch in range(12):
          i, d, id, y = img[batch], depth[batch], field_id[batch], Y[batch]
          i = torch.unsqueeze(i, 0)
          y = y.view(1, -1)
          d = d/1000

          optimizer.zero_grad()
          output = model(i)
          lossfn = nn.MSELoss()
          loss = torch.sqrt(lossfn(output.float(), y.float()))
          loss.backward()
          optimizer.step()          
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * 12, len(train_data),
                100. * batch_idx * 12 / len(train_data), loss.item()))                                                    

In [31]:
from torchvision import models
res18 = models.resnet18(pretrained=True)
numFit = res18.fc.in_features
res18.fc = nn.Linear(numFit, 12)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [32]:
res18.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
res18.to(device)
optimizer = torch.optim.SGD(res18.parameters(), lr=0.01, momentum=0.5)

In [34]:
for epoch in range(0, 1):
    train(epoch, res18, optimizer)

Train Epoch: 0 [0/3396 (0%)]	Loss: 0.335825
Train Epoch: 0 [120/3396 (4%)]	Loss: 0.296729
Train Epoch: 0 [240/3396 (7%)]	Loss: 0.270626
Train Epoch: 0 [360/3396 (11%)]	Loss: 0.284463
Train Epoch: 0 [480/3396 (14%)]	Loss: 0.299855
Train Epoch: 0 [600/3396 (18%)]	Loss: 0.208195
Train Epoch: 0 [720/3396 (21%)]	Loss: 0.211066
Train Epoch: 0 [840/3396 (25%)]	Loss: 0.186550
Train Epoch: 0 [960/3396 (28%)]	Loss: 0.179468
Train Epoch: 0 [1080/3396 (32%)]	Loss: 0.136200
Train Epoch: 0 [1200/3396 (35%)]	Loss: 0.138124
Train Epoch: 0 [1320/3396 (39%)]	Loss: 0.045594
Train Epoch: 0 [1440/3396 (42%)]	Loss: 0.073844
Train Epoch: 0 [1560/3396 (46%)]	Loss: 0.085577
Train Epoch: 0 [1680/3396 (49%)]	Loss: 0.083416
Train Epoch: 0 [1800/3396 (53%)]	Loss: 0.102006
Train Epoch: 0 [1920/3396 (57%)]	Loss: 0.056621
Train Epoch: 0 [2040/3396 (60%)]	Loss: 0.033475
Train Epoch: 0 [2160/3396 (64%)]	Loss: 0.043741
Train Epoch: 0 [2280/3396 (67%)]	Loss: 0.059746
Train Epoch: 0 [2400/3396 (71%)]	Loss: 0.051525
Train 

In [35]:
import csv
import pandas as pd

outfile = 'submission.csv'

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
preds = []


res18.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
with open(outfile, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(titles)
    for i in range(len(test_data)):
      if i % 10 == 0: print(i)
      img, depth, field_id = test_data.getTestitem(i)
      img= torch.unsqueeze(img, 0)
      output = res18(img)  
      output = output.tolist()
      output = output[0]
      output.insert(0, field_id)
      csvwriter.writerow(output)
print("Written to csv file {}".format(outfile))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
Written to csv file submission.csv
